In [0]:
%pip install azure-eventhub

In [0]:
%pip install azure-eventhub requests

In [0]:
%restart_python

In [0]:
import time
import json
import requests
from azure.eventhub import EventHubProducerClient, EventData

# ==========================================
# 1. CONFIGURATION
# ==========================================

# 🔴 PASTE YOUR CONNECTION STRING HERE
# It looks like: "Endpoint=sb://<namespace>.servicebus.windows.net/;SharedAccessKeyName=...;SharedAccessKey=..."
connection_string = "Endpoint=sb://YOUR_EVENT_HUB_NAMESPACE..."

# Ensure this matches your Event Hub Instance name exactly
event_hub_name = "earthquake-stream"

# Data Source (USGS Live Feed)
api_url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_hour.geojson"

# ==========================================
# 2. MAIN PRODUCER LOGIC
# ==========================================

def run_earthquake_producer():
    print(f"🚀 Starting Producer for Event Hub: {event_hub_name}")
    print("--------------------------------------------------")

    # Initialize the Client
    try:
        producer = EventHubProducerClient.from_connection_string(
            conn_str=connection_string,
            eventhub_name=event_hub_name
        )
    except Exception as e:
        print(f"❌ Connection Error: Check your connection string! Details: {e}")
        return

    with producer:
        while True:
            try:
                # --- Step A: Fetch Live Data ---
                response = requests.get(api_url)
                
                if response.status_code != 200:
                    print(f"⚠️ API Error (Status {response.status_code}). Retrying in 10s...")
                    time.sleep(10)
                    continue

                data = response.json()
                features = data.get('features', [])
                
                if not features:
                    print("💤 No recent earthquakes found. Waiting...")
                    time.sleep(60)
                    continue

                # --- Step B: Batch & Send Events ---
                event_data_batch = producer.create_batch()
                count = 0
                
                for feature in features:
                    # Convert single quake dictionary to JSON string
                    message_body = json.dumps(feature)
                    
                    try:
                        # Try adding to the current batch
                        event_data_batch.add(EventData(message_body))
                        count += 1
                    except ValueError:
                        # If batch is full (max size limit), send it immediately
                        producer.send_batch(event_data_batch)
                        print(f"   -> Batch limit reached. Sending intermediate batch...")
                        
                        # Start a new batch
                        event_data_batch = producer.create_batch()
                        event_data_batch.add(EventData(message_body))
                        count += 1

                # Send the remaining events in the final batch
                if count > 0:
                    producer.send_batch(event_data_batch)
                    
                    # Log success with timestamp
                    current_time = time.strftime("%H:%M:%S")
                    print(f"✅ [{current_time}] Successfully pushed {count} events to Azure Event Hubs.")

                # --- Step C: Rate Limiting ---
                # USGS updates every 60 seconds. We wait to avoid spamming duplicate data.
                time.sleep(60)

            except Exception as e:
                print(f"❌ CRITICAL ERROR: {e}")
                print("Re-attempting in 10 seconds...")
                time.sleep(10)

# ==========================================
# 3. EXECUTION
# ==========================================
if __name__ == "__main__":
    run_earthquake_producer()